# Exploración Inicial

## Información general

In [91]:
#Importamos las librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly_express as px

In [92]:
# lectura de los datos
df = pd.read_csv("/Users/emilianogtlp/Documents/proyectos/dsc_datathon_23/dataset_transacc_tdc_tdd_20230309.csv", parse_dates=['fecha_transaccion'])

In [93]:
df.head(5)

,tipo_transaccion,nombre_comercio,giro_nombre,mcc_nombre,fecha_transaccion,ope_pais,entry_mode,sexo_cliente,estado_cliente,edad_cliente,monto_transaccion,id_cliente
0,TDC,FARM GUADALAJARA 341 TORREON COAH CO,FARMACIAS,"DROGUERIAS, (FARMACIAS)",2023-01-20 13:48:00,MX,CARD PRESENT,F,COAHUILA DE ZARAGOZA,61.0,65.04,933
1,TDC,OXXO BABY O ACAPULCO GRO GE,RETAIL,"TIENDAS DE CONVENIENCIA, MINISUPER",2023-01-28 13:58:00,MX,CARD PRESENT,M,MEXICO,27.0,143.18,8562
2,TDD,VS TELCEL 018001200006CIUDAD DE MEX001,TELECOMUNICACIONES,"SERVICIOS DE TELECOMUNICACION, LLAMADAS LOC Y ...",2023-01-29 13:37:00,MX,CARD NOT PRESENT,M,QUINTANA ROO,29.0,84.80,3980
3,TDD,KFC ECOMMERCE CIUDAD DE MEX001,COMIDA RAPIDA,COMIDA RAPIDA,2023-02-11 15:06:00,MX,CARD NOT PRESENT,M,COAHUILA DE ZARAGOZA,45.0,124.22,9389
4,TDD,MERCADO PAGO CIUDAD DE MEXDF,OTROS,MERCADEO DIRECTO - NO CLASIFICADO,2023-01-24 12:50:00,MX,CARD NOT PRESENT,M,TABASCO,22.0,17.44,1845


In [94]:
df.shape

(358360, 12)

In [95]:
df.columns

Index(['tipo_transaccion', 'nombre_comercio', 'giro_nombre', 'mcc_nombre',
       'fecha_transaccion', 'ope_pais', 'entry_mode', 'sexo_cliente',
       'estado_cliente', 'edad_cliente', 'monto_transaccion', 'id_cliente'],
      dtype='object')

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358360 entries, 0 to 358359
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   tipo_transaccion   358360 non-null  object        
 1   nombre_comercio    358360 non-null  object        
 2   giro_nombre        358360 non-null  object        
 3   mcc_nombre         358360 non-null  object        
 4   fecha_transaccion  358360 non-null  datetime64[ns]
 5   ope_pais           358360 non-null  object        
 6   entry_mode         358360 non-null  object        
 7   sexo_cliente       358360 non-null  object        
 8   estado_cliente     358360 non-null  object        
 9   edad_cliente       358360 non-null  float64       
 10  monto_transaccion  358360 non-null  float64       
 11  id_cliente         358360 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 32.8+ MB


In [97]:
df['fecha_transaccion'].describe(datetime_is_numeric=True)

count                           358360
mean     2023-01-09 18:43:32.353554432
min                2022-11-08 00:01:00
25%                2022-12-10 18:25:45
50%                2023-01-10 20:48:00
75%                2023-02-09 15:20:00
max                2023-03-07 23:57:00
Name: fecha_transaccion, dtype: object

## Obtener información sobre cada columna

In [98]:
# Analizar el tipo de transacción más común
df.tipo_transaccion.value_counts()

TDD    209698
TDC    148662
Name: tipo_transaccion, dtype: int64

No existen errores a corregir en el método de pago

In [99]:
len(df.nombre_comercio.unique())

84774

In [100]:
# Analizar la concurrencia de los comercios
df.nombre_comercio.value_counts()

MERCADO PAGO          CIUDAD DE MEXDF     8816
STP*UBER PENDING      CIUDAD DE MEX001    6402
SNP*MERCADOPAGO       CIUDAD DE MEX001    6369
DLOCAL*DIDI RIDES     CIUDAD DE MEX001    4757
PAYPAL3               MEXICO DF    DF     2932
                                          ... 
SANBORNS SLP PZA CARSOSN LUIS POTOSSL        1
GALLARDO ORTODONCIA   MEXICALI     BC        1
GASOL TLACOTEPEC      TOLUCA EM    000       1
CHARLEYS PHILLY STEAKS 51SAN ANTONIO         1
FIRST CASH            SOLEDAD DE GR000       1
Name: nombre_comercio, Length: 84774, dtype: int64

Existen comercios con múltiples compras así como outliers, no se despreciaran ya que no se consideran una variable significante para el modelo

In [101]:
# Describir generalmente el compartamiento en cuanto a transacciones por giro
giros = df.groupby("giro_nombre").agg({'monto_transaccion':['count','sum','mean']})
giros_count = giros.sort_values(('monto_transaccion','count'),ascending=False)
print(giros_count)

                                  monto_transaccion                         
                                              count          sum        mean
giro_nombre                                                                 
RETAIL                                        69381   9898749.07  142.672332
SUPERMERCADOS                                 52479  11376961.52  216.790745
AGREGADOR                                     38739   6577058.73  169.778743
OTROS                                         36185   6673742.68  184.433956
TRANSPORTE TERRESTRE DE PASAJEROS             32118   2385990.48   74.288265
GASOLINERAS                                   20836   4764391.84  228.661540
RESTAURANTE                                   19632   4124162.52  210.073478
COMIDA RAPIDA                                 18314   2142755.16  117.000937
MISCELANEAS                                   15140   1720584.42  113.644942
ENTRETENIMIENTO                               14295   2738398.16  191.563355

In [102]:
# Lista de las categorías con mayor concurrencia
category_counts = df['giro_nombre'].value_counts(normalize=True).apply(lambda x: '{:.2f}%'.format(x*100))
category_counts


RETAIL                               19.36%
SUPERMERCADOS                        14.64%
AGREGADOR                            10.81%
OTROS                                10.10%
TRANSPORTE TERRESTRE DE PASAJEROS     8.96%
GASOLINERAS                           5.81%
RESTAURANTE                           5.48%
COMIDA RAPIDA                         5.11%
MISCELANEAS                           4.22%
ENTRETENIMIENTO                       3.99%
FARMACIAS                             3.48%
TELECOMUNICACIONES                    3.02%
GOBIERNO                              1.48%
REFACCIONES Y FERRETERIA              1.00%
MEDICOS Y DENTISTAS                   0.39%
HOTELES                               0.34%
COLEGIOS Y UNIVERSIDADES              0.34%
TRANSPORTE AEREO                      0.30%
ASEGURADORAS                          0.21%
SALONES DE BELLEZA                    0.19%
BENEFICENCIA                          0.15%
ESTACIONAMIENTOS                      0.14%
PEAJE                           

In [103]:
# Obtener suma de todas las transacciones
total_sum = df['monto_transaccion'].sum()

# Obtener suma de monto por giro asi como %
category_sums = df.groupby('giro_nombre')['monto_transaccion'].sum()
category_percentages = (category_sums / total_sum) * 100

# Organizar df de mayor a menor
category_percentages= category_percentages.sort_values(ascending=False).apply(lambda x: '{:.2f}%'.format(x))

print(category_percentages)

giro_nombre
SUPERMERCADOS                        18.50%
RETAIL                               16.10%
OTROS                                10.85%
AGREGADOR                            10.69%
GASOLINERAS                           7.75%
RESTAURANTE                           6.71%
ENTRETENIMIENTO                       4.45%
TRANSPORTE TERRESTRE DE PASAJEROS     3.88%
COMIDA RAPIDA                         3.48%
TELECOMUNICACIONES                    3.10%
MISCELANEAS                           2.80%
FARMACIAS                             2.46%
GOBIERNO                              2.18%
REFACCIONES Y FERRETERIA              1.44%
COLEGIOS Y UNIVERSIDADES              0.90%
TRANSPORTE AEREO                      0.89%
MEDICOS Y DENTISTAS                   0.84%
HOTELES                               0.72%
ASEGURADORAS                          0.57%
AGENCIAS DE VIAJES                    0.36%
EDUCACIN BASICA                       0.35%
BENEFICENCIA                          0.28%
SALONES DE BELLEZA  

In [104]:
# Volver a convertir a númerico para poder usar condicionales flotantes 
category_counts_numeric = category_counts.str.replace('%', '').astype(float) / 100
category_percentages_numeric = category_percentages.str.replace('%', '').astype(float) / 100

# Obtener listas de giros con atributos menores a 1%
category_counts_less_than_1pct = category_counts_numeric[category_counts_numeric < 0.01].index
category_percentages_less_than_1pct = category_percentages_numeric[category_percentages_numeric < 0.1].index

# Obtener el set con la interescción de ambas listas
categories_less_than_1pct= set(category_counts_less_than_1pct).intersection(category_percentages_less_than_1pct)

print(categories_less_than_1pct)

{'RENTA DE AUTOS', 'ASEGURADORAS', 'TRANSPORTE AEREO', 'HOTELES', 'COLEGIOS Y UNIVERSIDADES', 'AGENCIAS DE VIAJES', 'EDUCACIN BASICA', 'MEDICOS Y DENTISTAS', 'BENEFICENCIA', 'HOSPITALES', 'ESTACIONAMIENTOS', 'SALONES DE BELLEZA', 'PEAJE', 'GUARDERIAS'}


Las categorias que represtan menos del 1% tanto en ocurrencias como en cantidad transaccionada son:
* 'SALONES DE BELLEZA'
* 'ASEGURADORAS'
* 'HOSPITALES'
* 'ESTACIONAMIENTOS'
* 'HOTELES'
* 'TRANSPORTE AEREO'
* 'AGENCIAS DE VIAJES'
* 'RENTA DE AUTOS'
* 'PEAJE'
* 'COLEGIOS Y UNIVERSIDADES'
* 'EDUCACIN BASICA'
* 'BENEFICENCIA'
* 'GUARDERIAS'
* 'MEDICOS Y DENTISTAS'

Por lo tanto, dichos registros se podrían omitir por ser considerados outliers en un modelo

In [105]:
df.mcc_nombre.value_counts()

AGREGADOR                                             38739
AUTOSERVICIOS Y SUPERMERCADOS                         36300
TIENDAS DE CONVENIENCIA, MINISUPER                    34628
LIMOSINAS, (TAXIS)                                    30879
ESTACIONES DE SERVICIO, GASOLINERIAS TODO TRANSPOR    20760
                                                      ...  
COMFORT INNS                                              1
HOWARD JOHNSON INN, (HOWARD JOHNSON)                      1
DISTRIBUIDORES - CAMPERS Y REMOLQUES                      1
COLLECC DE MONEDAS, NUMISMATICA, ESTAMPILLAS              1
AUSTRIAN AIRLINES                                         1
Name: mcc_nombre, Length: 294, dtype: int64

Debido a su fuere relación con el giro, se desprecia cualquier hallazgo de esta columna

In [106]:
# Mostrar el número de ocurrencias de cada fecha
date_counts = df.groupby(pd.Grouper(key='fecha_transaccion', freq='D')).agg(count=('monto_transaccion', 'count'), total_amount=('monto_transaccion', 'sum'))
date_counts


,count,total_amount
fecha_transaccion,,
2022-11-08,2365,377732.70
2022-11-09,2382,357018.91
2022-11-10,2451,394385.45
2022-11-11,2712,443472.35
2022-11-12,2965,502732.06
...,...,...
2023-03-03,3711,604089.48
2023-03-04,4117,717221.31
2023-03-05,3494,613187.74


In [107]:
date_counts.describe()

,count,total_amount
count,120.000000,120.000000
mean,2986.333333,512493.840250
std,382.559741,76143.761476
min,1549.000000,256053.130000
25%,2721.000000,457908.127500
50%,2965.500000,506654.505000
75%,3214.000000,557369.877500
max,4117.000000,717221.310000


Los registros abarcan 120 días, del 8 de noviembre del 2022, al 7 de marzo del 2023.

El promedio de transacciones por día es de 2986, mientras que el valor promedio transaccionado por día es de 512,494

Debido a que cada día cuenta con un número considerable de datos, no se planea eliminar uno en particular

In [108]:
# Mostrar los paises con mayor monto_transaccionado y mayor número de transacciones
paises = df.groupby("ope_pais").agg({'monto_transaccion':['count','sum']}).sort_values(('monto_transaccion','sum'),ascending=False)
paises.head(10)

monto_transaccion             
                     count          sum
ope_pais                               
MX                  338768  58104366.23
US                    8818   1504670.54
GB                    3389    604020.79
NL                    2085    244950.33
ES                     810    156446.73
IE                     837    152468.13
LU                     494    111952.91
CA                     350     71794.13
LT                     102     48019.49
FR                     183     42002.00

Fuera de 'US' ningún otro país muestra relevancia para ser considerado en un modelo

In [109]:
df.entry_mode.value_counts()

CARD PRESENT        227208
CARD NOT PRESENT    131152
Name: entry_mode, dtype: int64

No existen valores atípicos en el método de pago

In [110]:
df.sexo_cliente.value_counts()

M    244682
F    113378
        300
Name: sexo_cliente, dtype: int64

Existen 300 registros que no tienen un sexo asignado, estos se pueden despreciar

In [111]:
# Obtener el % de transacciones por estado de domicilio de usuario
df.estado_cliente.value_counts(normalize=True)
#len(df.estado_cliente.unique())


NUEVO LEON                        0.222391
CIUDAD DE MEXICO                  0.144363
MEXICO                            0.100357
JALISCO                           0.079219
BAJA CALIFORNIA                   0.038645
QUERETARO                         0.034250
CHIHUAHUA                         0.032718
COAHUILA DE ZARAGOZA              0.032551
GUANAJUATO                        0.029331
VERACRUZ DE IGNACIO DE LA LLAV    0.027986
QUINTANA ROO                      0.024149
PUEBLA                            0.023532
YUCATAN                           0.021989
TAMAULIPAS                        0.021841
SONORA                            0.020708
SAN LUIS POTOSI                   0.018225
SINALOA                           0.013964
AGUASCALIENTES                    0.013514
MICHOACAN DE OCAMPO               0.012019
DURANGO                           0.011790
HIDALGO                           0.010763
MORELOS                           0.010682
TABASCO                           0.009256
BAJA CALIFO

Gran parte de las transacciones son hechas por usuarios con domicilio en el norte del país

In [112]:
df.edad_cliente.value_counts()

27.0    19220
31.0    17623
25.0    17403
26.0    17108
30.0    16997
        ...  
12.0       20
77.0       15
11.0        8
3.0         8
82.0        2
Name: edad_cliente, Length: 80, dtype: int64

In [113]:
df[['edad_cliente','monto_transaccion']].describe()

,edad_cliente,monto_transaccion
count,358360.000000,358360.000000
mean,33.647918,171.613073
std,11.956654,187.012943
min,1.000000,10.000000
25%,26.000000,48.370000
50%,31.000000,98.680000
75%,38.000000,224.270000
max,123.000000,5000.000000


In [114]:
len(df[df['edad_cliente']<18])

1709

In [115]:
len(df[df['edad_cliente']>85])

1901

Existen registros de menores de edad, esto puede parecer raro, pero debido a la emision de Hey, Menores, puede hacer creibles estas transacciones. Sin embargo, como representan menos del 1% de las transacciones totales se despreciaran para un modelo, al igual que los registros de personas mayores a 85 años

# Limpieza de datos

## Borrar datos atípicos

Existen diversos registros que siguen categorias de atributos que son despreciables para la elaboración de nuestro modelo:
- Registros con giros en : {'RENTA DE AUTOS', 'ASEGURADORAS', 'TRANSPORTE AEREO', 'HOTELES', 'COLEGIOS Y UNIVERSIDADES', 'AGENCIAS DE VIAJES', 'EDUCACIN BASICA', 'MEDICOS Y DENTISTAS', 'BENEFICENCIA', 'HOSPITALES', 'ESTACIONAMIENTOS', 'SALONES DE BELLEZA', 'PEAJE', 'GUARDERIAS'}
- Registros con país de operación diferente a: {'MX','US'}
- Registros con sexo del cliente indefinido
- Registros con edad de cliente menor a 18 o mayor a 85

Se procede a eliminar dichos registros

In [119]:
# Elmina todos los registros que contengan alguna de las categorías con menor a 1%
df = df[~df['giro_nombre'].isin(categories_less_than_1pct)]

# Eliminar todos los registros realizados en un país que no sea México o USA
df = df[df['ope_pais'].isin({'MX','US'})]

# Eliminar registro con sexo indefinido
df = df[df['sexo_cliente'].isin({'M','F'})]

# Eliminar registros de clientes con edad menor a 18 años o mayor a 85
df = df[(df['edad_cliente'] >= 18) & (df['edad_cliente'] <= 85)]

In [120]:
df

,tipo_transaccion,nombre_comercio,giro_nombre,mcc_nombre,fecha_transaccion,ope_pais,entry_mode,sexo_cliente,estado_cliente,edad_cliente,monto_transaccion,id_cliente
0,TDC,FARM GUADALAJARA 341 TORREON COAH CO,FARMACIAS,"DROGUERIAS, (FARMACIAS)",2023-01-20 13:48:00,MX,CARD PRESENT,F,COAHUILA DE ZARAGOZA,61.0,65.04,933
1,TDC,OXXO BABY O ACAPULCO GRO GE,RETAIL,"TIENDAS DE CONVENIENCIA, MINISUPER",2023-01-28 13:58:00,MX,CARD PRESENT,M,MEXICO,27.0,143.18,8562
2,TDD,VS TELCEL 018001200006CIUDAD DE MEX001,TELECOMUNICACIONES,"SERVICIOS DE TELECOMUNICACION, LLAMADAS LOC Y ...",2023-01-29 13:37:00,MX,CARD NOT PRESENT,M,QUINTANA ROO,29.0,84.80,3980
3,TDD,KFC ECOMMERCE CIUDAD DE MEX001,COMIDA RAPIDA,COMIDA RAPIDA,2023-02-11 15:06:00,MX,CARD NOT PRESENT,M,COAHUILA DE ZARAGOZA,45.0,124.22,9389
4,TDD,MERCADO PAGO CIUDAD DE MEXDF,OTROS,MERCADEO DIRECTO - NO CLASIFICADO,2023-01-24 12:50:00,MX,CARD NOT PRESENT,M,TABASCO,22.0,17.44,1845
...,...,...,...,...,...,...,...,...,...,...,...,...
358355,TDD,SNP*MERCADOPAGO CIUDAD DE MEX001,AGREGADOR,AGREGADOR,2023-01-13 14:36:00,MX,CARD NOT PRESENT,F,VERACRUZ DE IGNACIO DE LA LLAV,23.0,531.91,463
358356,TDD,WALMART SUPER 1 CIUDAD DE MEXDF,SUPERMERCADOS,AUTOSERVICIOS Y SUPERMERCADOS,2022-11-27 02:22:00,MX,CARD NOT PRESENT,F,CIUDAD DE MEXICO,46.0,531.24,4167
358357,TDC,CLIP MX*TIJUANA SMOKE TIJUANA 000,AGREGADOR,AGREGADOR,2023-02-27 15:35:00,MX,CARD PRESENT,M,BAJA CALIFORNIA,21.0,523.99,3264
358358,TDC,12746 ZARA PASEOQUERE QUERETARO 00,RETAIL,ROPA CASUAL DAMAS LENCERIA,2022-12-09 18:28:00,MX,CARD PRESENT,F,QUERETARO,23.0,548.17,8180


Se eliminaron 22,613 registros atípicos, conformando el 6% del data set original

## Generación de clases

In [121]:
# Agrupamos los datos en base al área de procedencia del usuario

In [122]:
# Creamo grupos para los distintos rangos de edad de los usuarios

In [123]:
# Creamos grupos para el monto de las transacciones